In [1]:
import os
import sys
import subprocess as sp
import json
import random
import time
import hashlib
import base64
from contextlib import closing
from dataclasses import dataclass
import typing

import multiprocessing
# from multiprocessing import Barrier

from concurrent.futures.thread import ThreadPoolExecutor
from concurrent.futures.process import ProcessPoolExecutor

import psycopg2
import psycopg2.extras

### Get a quick pg instance:
---

```bash
# ephemeral pg13
docker run --rm -it --name pg13 --network host -e POSTGRES_PASSWORD=bobz1234 postgres:13 --port=9932


# put data under tmpfs
# PGDATA tells it where to put data, needs to be someplace that can be chowned to postgres user
# by default things should be under /var/lib/postgresql/data
docker run --rm -it --name pg13 --network host -e POSTGRES_PASSWORD=bobz1234 \
-e PGDATA=/var/lib/postgresql/data/tmpfs_vol/pgdata \ 
--tmpfs /var/lib/postgresql/data/tmpfs_vol/ postgres:13 --port=9932 --listen-addresses='*'


# if connecting from remote add
--listen-addresses='*'
```

In [2]:

# dsn = "... dbname=postgres options='-c synchronous_commit=off'"
_PG_DSN_SYNC = {
    # "host": "192.168.24.141",
    # "port": "5432",
    
    # "host": "127.0.0.1",
    "host": "192.168.24.140",
    "port": "9932",
    
    "dbname": "postgres",
    "user": "postgres",
    "password": 'bobz1234',
    
    # any other options should be passed along to underlying lib, but only those psycopg2 understands
    # and accepts. "nonz": "hello world" wouldnt be accepted or passed along
    "application_name": "pg_bench_ntbk",
}

_PG_DSN_NO_SYNC = _PG_DSN_SYNC.copy() # shallow is enof.
_PG_DSN_NO_SYNC["options"] = "-c synchronous_commit=off"
_PG_DSN_NO_SYNC["application_name"] = "fts_ntbk_unsync"

In [ ]:
sync_dbconn = psycopg2.connect(**_PG_DSN_SYNC)
sync_dbconn.autocommit = True

unsync_dbconn = psycopg2.connect(**_PG_DSN_NO_SYNC)
unsync_dbconn.autocommit = True

In [4]:
sync_cur = sync_dbconn.cursor()
unsync_cur = unsync_dbconn.cursor()

In [5]:
# common q exec.
def exec_q_unsync(q, pr_res=True, col_names=True):
    with closing(unsync_dbconn.cursor()) as tmp_cur:
        tmp_cur.execute(q)
        if pr_res:
            if col_names:
                print("|".join([desc[0] for desc in tmp_cur.description]))
            print('-'*40)
            for row in tmp_cur.fetchall():
                print(row)
            # sep
            print("")
# sync
def exec_q_sync(q, pr_res=True, col_names=True):
    with closing(sync_dbconn.cursor()) as tmp_cur:
        tmp_cur.execute(q)
        if pr_res:
            if col_names:
                print("|".join([desc[0] for desc in tmp_cur.description]))
            print('-'*40)
            for row in tmp_cur.fetchall():
                print(row)
            # sep
            print("")

# Run some test queries

In [6]:
exec_q_sync("SHOW synchronous_commit;")
exec_q_unsync("SHOW synchronous_commit;")

synchronous_commit
----------------------------------------
('on',)

synchronous_commit
----------------------------------------
('off',)



# Generate sample 

In [7]:
# int32 is usually ranged: -2 147 483 648 to 2 147 483 647
def get_random_int32():
    tmp = 2 * 1000 * 1000 * 1000
    return random.randint(0, tmp)

def get_random_int64():
    tmp = 80 * 1000 * 1000 * 1000 * 1000
    return random.randint(0, tmp)

def get_random_str(str_len=48):
    assert isinstance(str_len, int)
    assert str_len < 8192
    return base64.b64encode(os.urandom(str_len), altchars=b"AZ").decode('ascii')

In [8]:
# demo records
for _ in range(5):
    print((get_random_str(48), get_random_int32()))



('9prwcUq3EEoDB3B8fufKLUBoZbuaRvQHQxFLrgASYMvvn4j1UwnzKZQW0xT76H38', 18670438)
('jpe4X8e5ILngFKOkMD7unSoBUtdagjOD0KA3Zm2UTFu6Z6AZQgyum3DUzQVcI8Br', 116052191)
('EEsGwvDwuVoP6A8VbowRKJnHK3tF162pZZighZBU2gASorOrSc7xWo5FPzsj4OYl', 230162896)
('qiGCy1NotC99oC7VtZbzbJRWfoiZx2wA1Rof1BWcZdJHCfvu7kZmCcAyZYNbZpAd', 1015894051)
('IYDvZ0UjD45eGgnOoKSW0pPimImPMhiwCZJIeTGuLAWZOo5fLzj7c0EOmuPZ1uSj', 900492258)


# Create BENCHMARK tables

In [9]:
refresh_schema_q = """
DROP TABLE IF EXISTS xb_bench1 CASCADE;
DROP TABLE IF EXISTS xb_bench2 CASCADE;

CREATE TABLE IF NOT EXISTS xb_bench1(
brid SERIAL PRIMARY KEY NOT NULL,
val_1 TEXT,
num_1 BIGINT);


CREATE TABLE IF NOT EXISTS xb_bench2(
brid SERIAL PRIMARY KEY NOT NULL,
val_1 TEXT,
num_1 BIGINT);
"""

In [10]:
exec_q_sync(refresh_schema_q, pr_res=False)

In [11]:
# test insert some records

In [13]:
q = """ INSERT INTO xb_bench1(val_1, num_1) VALUES 
('Q8M5rzhQveJJClZAGhXtNmWCgRm2ETnSeZb2KdXy1vDVlnj3tYOlOZulh2Fc7T1p', '2120724'),
('EEsGwvDwuVoP6A8VbowRKJnHK3tF162pZZighZBU2gASorOrSc7xWo5FPzsj4OYl', '2020921');
"""

sync_cur.execute(q)
exec_q_sync("select * from xb_bench1;")
exec_q_sync(refresh_schema_q, pr_res=False)

print('db clean again:')
exec_q_sync("select * from xb_bench1;")

brid|val_1|num_1
----------------------------------------
(1, 'Q8M5rzhQveJJClZAGhXtNmWCgRm2ETnSeZb2KdXy1vDVlnj3tYOlOZulh2Fc7T1p', 2120724)
(2, 'EEsGwvDwuVoP6A8VbowRKJnHK3tF162pZZighZBU2gASorOrSc7xWo5FPzsj4OYl', 2020921)

db clean again:
brid|val_1|num_1
----------------------------------------



---
---
---
---
---
# Concurrent/Parallel Benchmarks:


In [14]:
@dataclass(frozen=True)
class BenchParams:
    wrkr_id: str
    pg_dsn_kwargs: dict
    start_barrier: typing.Any
    
    # benchmark 4 ips (insertion per sec)
    b4i_num_records: int


In [15]:
@dataclass(frozen=True)
class BenchResult:
    wrkr_id: str
    
    wrkr_start_time: float
    wrkr_end_time: float
    
    # benchmark for ips
    b4i_start_time: float
    b4i_end_time: float
    
    b4i_num_records: int
    b4i_ips: int
    
    def __str__(self):
        
        _deltaT = self.b4i_end_time - self.b4i_start_time
        _ips = int(self.b4i_num_records / _deltaT)
        _b4i_start = self.b4i_start_time
        
        # verbose
#         res =  f"""\n{"=" * 80} {self.wrkr_id}:
# b4i start  : {self.b4i_start_time}
# b4i end    : {self.b4i_end_time}
# b4i deltaT : {_deltaT:.4f}
# b4i num records : {self.b4i_num_records:,}
# ips             : {_ips}"""

        res = f"wrkr_id: {self.wrkr_id} -- b4i_start: {_b4i_start:,.4f} -- deltaT: {_deltaT:.4f} -- ips: {_ips}" 
        return res

In [16]:
class IPS_Benchmark:
    def __init__(self, bp: BenchParams):
        
        self.bp = bp
        
        # ----- generate sample data:
        self.b4i_data = []
        for _ in range(bp.b4i_num_records):
            # almost no diff between 48 or just 4 in ips
            tmp_record = (get_random_str(48), get_random_int32())
            self.b4i_data.append(tmp_record)
        
        # ----- connect to db        
        self.db_conn = psycopg2.connect(**bp.pg_dsn_kwargs)
        self.db_conn.autocommit = True
        
        print(f"{bp.wrkr_id}: init compelete. time: {time.time():,.5f}\n", end="")
        
    def run_b4i(self):
        
        with self.db_conn.cursor() as tmp_cur:
            _q = "INSERT INTO xb_bench1(val_1, num_1) VALUES %s;"
            psycopg2.extras.execute_values(tmp_cur, _q, self.b4i_data)
        
    

In [17]:
def wrkr_entry(bp: BenchParams):
    
    _wrkr_start_time = time.time()
    
    # print(f"{bp.wrkr_id}: wrkr_entry() called. Time: {_wrkr_start_time:,.5f}\n", end="")
    
    bench = IPS_Benchmark(bp)
    
    # comment/uncomment sleep to simulate additional init delay
    # time.sleep(1)
    
    # synchronize all benchmarks to start at the same time. ie using a barrier
    bp.start_barrier.wait()
    
    print(f"{bp.wrkr_id} awoken. worker is passed barrier. time: {time.time():,.5f}\n", end="")
    
    _b4i_start_time = time.time()
    bench.run_b4i()
    _b4i_end_time = time.time()
    
    # calculate some benchmark stats for the returning result
    _b4i_num_records = len(bench.b4i_data)
    _b4i_ips = _b4i_num_records / (_b4i_end_time - _b4i_start_time)
    
    
    bres = BenchResult(wrkr_id=bp.wrkr_id, wrkr_start_time=_wrkr_start_time, wrkr_end_time=time.time(),
                       b4i_start_time=_b4i_start_time, b4i_end_time=_b4i_end_time,
                       b4i_num_records=_b4i_num_records, b4i_ips=_b4i_ips)
    
    return bres

---
---
---
# ProcessPoolExecutor


In [23]:
# ------------------------------------------------ lets go w/ option1: map and iterate results
# option 2 would be
# futures.append( ... submit() ...)
# wait(futures)

# ----------------------------------------- benchmark knobs
ppx_worker_count = 4
ppx_num_records = 400 * 1000

# prep db 
with sync_dbconn.cursor() as tmp_cur:
    tmp_cur.execute(refresh_schema_q)
print(f"db is clean. benchamrk rdy to run ...\n")

mgr = multiprocessing.Manager()
ppx_synd_start = mgr.Barrier(ppx_worker_count)
# ppx_synd_start = None


worker_args = []
for i in range(ppx_worker_count):
    _bp = BenchParams(wrkr_id=f'w{i}', b4i_num_records=ppx_num_records, pg_dsn_kwargs=_PG_DSN_SYNC,
                      start_barrier=ppx_synd_start)
    worker_args.append(_bp)

# results
ppx_waited_results = []

# make ppx and run 
# master time is worthless, we are generating mock data in init, dont measure it, needless complexity
with ProcessPoolExecutor(max_workers=ppx_worker_count) as ppx:
    map_res_iterator = ppx.map(wrkr_entry, worker_args)
    for res in map_res_iterator:
        ppx_waited_results.append(res)

# 
print("")
print('# ' + '-' * 97)
print('# ' + '-' * 97)

print('=' * 20 + " benchmark results: ")
total_inserted_records = sum([res.b4i_num_records for res in ppx_waited_results])
b4i_workers_ips_sum = int(sum([res.b4i_ips for res in ppx_waited_results]))

# (includes record generation, db connect, sync, ...)
print(f"num wrkrs: {ppx_worker_count}")
print(f"records per wrkr: {ppx_num_records:,}")

with unsync_dbconn.cursor() as tmp_cur:
    tmp_cur.execute('SELECT count(*) FROM xb_bench1;')
    print(f"Actual records in xb_bench1 table: {tmp_cur.fetchone()[0]:,}")
    
    # drop tables so it doesnt go and try to resolve WAL and what not.
    tmp_cur.execute(refresh_schema_q)

# 
print(f"\n*** Sum of syncd wrkr ips  : {b4i_workers_ips_sum:,}")

print('\n' + '=' * 20 + " worker info: ")
for res in ppx_waited_results:
    print(res)

db is clean. benchamrk rdy to run ...

w3: init compelete. time: 1,604,576,156.52969
w2: init compelete. time: 1,604,576,156.53489
w0: init compelete. time: 1,604,576,156.54012
w1: init compelete. time: 1,604,576,156.55390
w1 awoken. worker is passed barrier. time: 1,604,576,156.55600
w2 awoken. worker is passed barrier. time: 1,604,576,156.55617
w3 awoken. worker is passed barrier. time: 1,604,576,156.55623
w0 awoken. worker is passed barrier. time: 1,604,576,156.55613

# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
==================== benchmark results: 
num wrkrs: 4
records per wrkr: 400,000
Actual records in xb_bench1 table: 1,600,000

*** Sum of syncd wrkr ips  : 234,431

==================== worker info: 
wrkr_id: w0 -- b4i_start: 1,604,576,156.5573 -- deltaT: 6.8260 -- ips: 58599
wrkr_id: w1 -- b4i_start: 1,604,576,156.5571 -- 

```text

# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
==================== benchmark results: 
num wrkrs: 8
records per wrkr: 400,000
Actual records in xb_bench1 table: 3,200,000

*** Sum of syncd wrkr ips  : 320,316

==================== worker info: 
wrkr_id: w0 -- b4i_start: 1,604,575,980.4382 -- deltaT: 9.9389 -- ips: 40245
wrkr_id: w1 -- b4i_start: 1,604,575,980.4372 -- deltaT: 9.9904 -- ips: 40038
wrkr_id: w2 -- b4i_start: 1,604,575,980.4376 -- deltaT: 10.0553 -- ips: 39779
wrkr_id: w3 -- b4i_start: 1,604,575,980.4387 -- deltaT: 9.9548 -- ips: 40181
wrkr_id: w4 -- b4i_start: 1,604,575,980.4381 -- deltaT: 9.9270 -- ips: 40294
wrkr_id: w5 -- b4i_start: 1,604,575,980.4368 -- deltaT: 10.0715 -- ips: 39715
wrkr_id: w6 -- b4i_start: 1,604,575,980.4370 -- deltaT: 10.0250 -- ips: 39900
wrkr_id: w7 -- b4i_start: 1,604,575,980.4378 -- deltaT: 9.9601 -- ips: 40160


- network usage: 28-30 MB/s stable


# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
==================== benchmark results: 
num wrkrs: 16
records per wrkr: 400,000
Actual records in xb_bench1 table: 6,400,000

*** Sum of syncd wrkr ips  : 309,729

==================== worker info: 
wrkr_id: w0 -- b4i_start: 1,604,576,077.8092 -- deltaT: 20.6500 -- ips: 19370
wrkr_id: w1 -- b4i_start: 1,604,576,077.8071 -- deltaT: 20.6666 -- ips: 19354
wrkr_id: w2 -- b4i_start: 1,604,576,077.8116 -- deltaT: 20.6118 -- ips: 19406
wrkr_id: w3 -- b4i_start: 1,604,576,077.8108 -- deltaT: 20.6829 -- ips: 19339
wrkr_id: w4 -- b4i_start: 1,604,576,077.8094 -- deltaT: 20.5849 -- ips: 19431
wrkr_id: w5 -- b4i_start: 1,604,576,077.8092 -- deltaT: 20.8545 -- ips: 19180
.... trunc ...

- network usage: 28-30 MB/s stable


# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
==================== benchmark results: 
num wrkrs: 4
records per wrkr: 400,000
Actual records in xb_bench1 table: 1,600,000

*** Sum of syncd wrkr ips  : 234,431

==================== worker info: 
wrkr_id: w0 -- b4i_start: 1,604,576,156.5573 -- deltaT: 6.8260 -- ips: 58599
wrkr_id: w1 -- b4i_start: 1,604,576,156.5571 -- deltaT: 6.8454 -- ips: 58433
wrkr_id: w2 -- b4i_start: 1,604,576,156.5573 -- deltaT: 6.7976 -- ips: 58844
wrkr_id: w3 -- b4i_start: 1,604,576,156.5574 -- deltaT: 6.8313 -- ips: 58554

- network usage: ~ 20 MB/s stable



# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------



# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------



# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------



# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------



# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------








```

---
# PPX Results Dump

```text

# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
# ---------------------------------------------- Ryzen 2700X, 32GB RAM, pg13 on dkr on tmpfs PGDATA


# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
==================== benchmark results: 
num wrkrs: 16
records per wrkr: 400,000
Actual records in xb_bench1 table: 6,400,000

*** Sum of syncd wrkr ips  : 341,944

==================== worker info: 
wrkr_id: w0 -- b4i_start: 1,604,456,922.3961 -- deltaT: 18.6815 -- ips: 21411
wrkr_id: w1 -- b4i_start: 1,604,456,922.3926 -- deltaT: 18.7718 -- ips: 21308
wrkr_id: w2 -- b4i_start: 1,604,456,922.3938 -- deltaT: 18.6393 -- ips: 21459
wrkr_id: w3 -- b4i_start: 1,604,456,922.3929 -- deltaT: 18.8011 -- ips: 21275
wrkr_id: w4 -- b4i_start: 1,604,456,922.3951 -- deltaT: 18.5248 -- ips: 21592
wrkr_id: w5 -- b4i_start: 1,604,456,922.3929 -- deltaT: 18.6606 -- ips: 21435
wrkr_id: w6 -- b4i_start: 1,604,456,922.3936 -- deltaT: 18.7656 -- ips: 21315
wrkr_id: w7 -- b4i_start: 1,604,456,922.3916 -- deltaT: 18.8446 -- ips: 21226
wrkr_id: w8 -- b4i_start: 1,604,456,922.3922 -- deltaT: 18.7874 -- ips: 21290
wrkr_id: w9 -- b4i_start: 1,604,456,922.3949 -- deltaT: 18.5196 -- ips: 21598
wrkr_id: w10 -- b4i_start: 1,604,456,922.3963 -- deltaT: 18.8223 -- ips: 21251
wrkr_id: w11 -- b4i_start: 1,604,456,922.3973 -- deltaT: 18.7661 -- ips: 21314
wrkr_id: w12 -- b4i_start: 1,604,456,922.3951 -- deltaT: 18.6571 -- ips: 21439
wrkr_id: w13 -- b4i_start: 1,604,456,922.3931 -- deltaT: 18.7664 -- ips: 21314
wrkr_id: w14 -- b4i_start: 1,604,456,922.3943 -- deltaT: 18.7043 -- ips: 21385
wrkr_id: w15 -- b4i_start: 1,604,456,922.3961 -- deltaT: 18.7592 -- ips: 21322

# NOTES:
- quite a bit worse ips than 8 workers
- it actually never used more than 450% CPU i suspect because there
wasnt any more avaialble and there was less cpu available compared
to 8 or 10 workers.

# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
==================== benchmark results:
num wrkrs: 10
records per wrkr: 400,000
Actual records in xb_bench1 table: 4,000,000

*** Sum of syncd wrkr ips  : 428,084

==================== worker info: 
wrkr_id: w0  ---  b4i_start: 1,604,455,588.1164  ---  deltaT: 9.3211  ---  ips: 42913
wrkr_id: w1  ---  b4i_start: 1,604,455,588.1185  ---  deltaT: 9.3788  ---  ips: 42649
wrkr_id: w2  ---  b4i_start: 1,604,455,588.1177  ---  deltaT: 9.3515  ---  ips: 42773
wrkr_id: w3  ---  b4i_start: 1,604,455,588.1163  ---  deltaT: 9.3592  ---  ips: 42738
wrkr_id: w4  ---  b4i_start: 1,604,455,588.1188  ---  deltaT: 9.3389  ---  ips: 42831
wrkr_id: w5  ---  b4i_start: 1,604,455,588.1169  ---  deltaT: 9.3821  ---  ips: 42634
wrkr_id: w6  ---  b4i_start: 1,604,455,588.1173  ---  deltaT: 9.3661  ---  ips: 42707
wrkr_id: w7  ---  b4i_start: 1,604,455,588.1180  ---  deltaT: 9.3182  ---  ips: 42926
wrkr_id: w8  ---  b4i_start: 1,604,455,588.1176  ---  deltaT: 9.3024  ---  ips: 42999
wrkr_id: w9  ---  b4i_start: 1,604,455,588.1189  ---  deltaT: 9.3220  ---  ips: 42909

# NOTES:
- pg container used up to 500% CPU, and I suspect there wasnt any more.



# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
==================== benchmark results: 
num wrkrs: 8
records per wrkr: 400,000
Actual records in xb_bench1 table: 3,200,000

*** Sum of syncd wrkr ips  : 439,998

==================== worker info: 
wrkr_id: w0  ---  b4i_start: 1,604,455,719.8974  ---  deltaT: 7.2736  ---  ips: 54993
wrkr_id: w1  ---  b4i_start: 1,604,455,719.8969  ---  deltaT: 7.2979  ---  ips: 54810
wrkr_id: w2  ---  b4i_start: 1,604,455,719.8978  ---  deltaT: 7.2715  ---  ips: 55009
wrkr_id: w3  ---  b4i_start: 1,604,455,719.8976  ---  deltaT: 7.2848  ---  ips: 54908
wrkr_id: w4  ---  b4i_start: 1,604,455,719.8979  ---  deltaT: 7.2146  ---  ips: 55443
wrkr_id: w5  ---  b4i_start: 1,604,455,719.8967  ---  deltaT: 7.2641  ---  ips: 55065
wrkr_id: w6  ---  b4i_start: 1,604,455,719.8980  ---  deltaT: 7.2996  ---  ips: 54797
wrkr_id: w7  ---  b4i_start: 1,604,455,719.8986  ---  deltaT: 7.2766  ---  ips: 54970


# NOTES:
- pg container used up to 470% CPU



# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
==================== benchmark results: 
num wrkrs: 6
records per wrkr: 400,000
Actual records in xb_bench1 table: 2,400,000

*** Sum of syncd wrkr ips  : 418,291

==================== worker info: 
wrkr_id: w0  ---  b4i_start: 1,604,455,976.3896  ---  deltaT: 5.7290  ---  ips: 69820
wrkr_id: w1  ---  b4i_start: 1,604,455,976.3893  ---  deltaT: 5.7637  ---  ips: 69399
wrkr_id: w2  ---  b4i_start: 1,604,455,976.3891  ---  deltaT: 5.7268  ---  ips: 69846
wrkr_id: w3  ---  b4i_start: 1,604,455,976.3902  ---  deltaT: 5.7965  ---  ips: 69006
wrkr_id: w4  ---  b4i_start: 1,604,455,976.3906  ---  deltaT: 5.7159  ---  ips: 69980
wrkr_id: w5  ---  b4i_start: 1,604,455,976.3902  ---  deltaT: 5.6951  ---  ips: 70236

# NOTES:
- pg container used up to 380% CPU



# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
==================== benchmark results: 
num wrkrs: 4
records per wrkr: 400,000
Actual records in xb_bench1 table: 1,600,000

*** Sum of syncd wrkr ips  : 347,043

==================== worker info: 
wrkr_id: w0  ---  b4i_start: 1,604,456,474.8659  ---  deltaT: 4.5433  ---  ips: 88042
wrkr_id: w1  ---  b4i_start: 1,604,456,474.8662  ---  deltaT: 4.6447  ---  ips: 86120
wrkr_id: w2  ---  b4i_start: 1,604,456,474.8653  ---  deltaT: 4.6783  ---  ips: 85500
wrkr_id: w3  ---  b4i_start: 1,604,456,474.8663  ---  deltaT: 4.5777  ---  ips: 87380

# NOTES:
- pg container used up to 260% CPU



# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
==================== benchmark results: 
num wrkrs: 3
records per wrkr: 400,000
Actual records in xb_bench1 table: 1,200,000

*** Sum of syncd wrkr ips  : 291,938

==================== worker info: 
wrkr_id: w0 -- b4i_start: 1,604,456,678.3362 -- deltaT: 4.1210 -- ips: 97063
wrkr_id: w1 -- b4i_start: 1,604,456,678.3367 -- deltaT: 4.0900 -- ips: 97798
wrkr_id: w2 -- b4i_start: 1,604,456,678.3368 -- deltaT: 4.1205 -- ips: 97076

# NOTES:
- pg container used up to 200% CPU



# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
==================== benchmark results: 
num wrkrs: 2
records per wrkr: 400,000
Actual records in xb_bench1 table: 800,000

*** Sum of syncd wrkr ips  : 200,876

==================== worker info: 
wrkr_id: w0 -- b4i_start: 1,604,456,723.3091 -- deltaT: 3.8668 -- ips: 103444
wrkr_id: w1 -- b4i_start: 1,604,456,723.3093 -- deltaT: 4.1055 -- ips: 97431

# NOTES:
- pg container used up to  133% CPU



# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
==================== benchmark results: 
num wrkrs: 1
records per wrkr: 400,000
Actual records in xb_bench1 table: 400,000

*** Sum of syncd wrkr ips  : 106,397

==================== worker info: 
wrkr_id: w0 -- b4i_start: 1,604,456,738.2717 -- deltaT: 3.7595 -- ips: 106397

# NOTES:
- pg container used up to  65% CPU



```

---
# PPX results over network

- client: ryzen 2700x, 32GB ram, .... gigabit network
- server: 8 cores vm ryzen 1700x, 12GB ram gigabit network

---
dsdsa


## ThreadPoolExeccutor